## Imports

In [3]:
import pypsa
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os


c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(


## Read network file

In [4]:
# Function to read in the network file

def read_network_file():
    network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_27_ec_lcopt_Co2L-1H.nc'
    n = pypsa.Network(network)
    return n

network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_27_ec_lcopt_Co2L-1H.nc'
n = pypsa.Network(network)

# folder where the results from the previous year is saved
scen_folder = 'results_RC_10'
scen = 'RC_10'
                                                                                                                                                                                                                                                                                                                               
# # Next year of the run
year = 2022



c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

## Scaling factors and Decomissioning

The scaling factors are used to increase the demand and decrease the emission constraint. The decomissioning (and comissioning, if applicable) is added on a yearly basis as well.

In [30]:
# Demand scaling factor - based on the initial demand in 2021

scale_demand = {
    2022: 1.039555791,
    2023: 1.037440228,
    2024: 1.030106894,
    2025: 1.029322163,
    2026: 1.051886792,
    2027: 1.044491339,
    2028: 1.044448186,
    2029: 1.044491013,
    2030: 1.044447874,
    2031: 1.044477281,
    2032: 1.044422438,
    2033: 1.040027091,
    2034: 1.039984371,
    2035: 1.040012523,
    'back':0.569269673, # scaling factor for scaling from 2035 to 2021
    'direct' : 1.7566367 # scaling factor from 2021 to 2035
}


In [4]:
# emissions decline

start_year = 2021
start_value = 2370000
end_year = 2035
end_value = 0

years = range(start_year, end_year + 1)
emissions = []

decrease_per_year = (start_value - end_value) / (end_year - start_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - start_year), end_value)
    emissions.append(emission)

# Make into dict
index = np.arange(2021,2036)
emission_limit = dict(zip(index, emissions))
emission_limit


{2021: 2370000.0,
 2022: 2200714.285714286,
 2023: 2031428.5714285714,
 2024: 1862142.8571428573,
 2025: 1692857.1428571427,
 2026: 1523571.4285714286,
 2027: 1354285.7142857143,
 2028: 1185000.0,
 2029: 1015714.2857142857,
 2030: 846428.5714285714,
 2031: 677142.857142857,
 2032: 507857.1428571427,
 2033: 338571.42857142864,
 2034: 169285.7142857141,
 2035: 0.0}

In [5]:
# decomissioning of powerplants

decom = {
2022 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.3, 'BO 2 OCGT': 36.76, 'BO 3 oil':0},
2023 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 36.77, 'BO 3 oil':0},
2024 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2025 : {'BO 0 OCGT':25.44, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2026 : {'BO 0 OCGT':0, 'BO 1 OCGT':17.86, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2027 : {'BO 0 OCGT':0, 'BO 1 OCGT':105.87, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2028 : {'BO 0 OCGT':0, 'BO 1 OCGT':16.9, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2029 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2030 : {'BO 0 OCGT':0, 'BO 1 OCGT':97.85, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2031 : {'BO 0 OCGT':0, 'BO 1 OCGT':109.37, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2032 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2033 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':2.16},
2034 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2035 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
}

# To force expansion, this dicts is used
expansion = {
    2022 : {'BO 0 geothermal':5, 'BO 0 solar':13, 'BO 2 ror':126},
    2023: { 'BO 2 solar':13, 'BO 3 ror':126},
    2024: {'BO 1 biomass': 21, 'BO 1 onwind':53, 'BO 0 solar':13, 'BO 2 ror':126},
    2025: { 'BO 1 onwind':24,'BO 2 solar':13, 'BO 3 ror':126},
    2026: {'BO 3 biomass': 21, 'BO 0 solar':13, 'BO 2 ror':126},
    2027: {'BO 0 geothermal':45, 'BO 1 onwind':25,'BO 2 solar':13, 'BO 3 ror':126},
    2028: {'BO 1 biomass': 21, 'BO 1 onwind':25, 'BO 0 solar':13, 'BO 2 ror':126},
    2029: {'BO 2 solar':13, 'BO 3 ror':126},
    2030: {'BO 3 biomass': 21, 'BO 1 onwind':35, 'BO 0 solar':13, 'BO 2 ror':126},
    2031: {},
    2032:{},
    2033:{},
    2034:{},
    2035:{},
}


# Nodes in the model (can change!!!!)
#0 : Southern
# 1: Oriental
#2: Central
#3: Northern

In [24]:
n.generators

#
# OCGT -> CCGT

{'p_min_pu': Empty DataFrame
 Columns: []
 Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013

## All changes

This cell makes all changes that need to be done to run the next year all at once and saves it in a new file. 

In [31]:
# Function that implements all yearly changes

def yearly_changes(n,year):
    print(year)
    # ------- DEMAND ---------
    upscaling_factor = scale_demand[year]
    n.loads_t.p_set = n.loads_t.p_set * upscaling_factor
    #display(round((n.loads_t.p_set.sum().sum()/1000000),4))

    # ------- EMISSIONS -------
    #n.global_constraints.constant = emission_limit[year]
    #display(n.global_constraints.constant)



    # ------- GENERATOR EXTENSTION -----
    solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/{scen}_{year-1}.nc'
    m = pypsa.Network(solved_network)
    additional_exp= m.generators.p_nom_opt - m.generators.p_nom
    # replace negative values with 0
    for index, value in additional_exp.items():
        if value < 0:
            additional_exp[index]=0
    # add expansion to previous network
    n.generators.p_nom = n.generators.p_nom.add(additional_exp, fill_value=0)
    n.generators.p_nom_min = n.generators.p_nom_min.add(additional_exp, fill_value=0)
    #display(n.generators.p_nom)


    # ------- STORES ----------
    additional_stores = m.stores.e_nom_opt - m.stores.e_nom
    additional_stores
    n.stores.e_nom = n.stores.e_nom.add(additional_stores, fill_value = 0)
    n.stores.e_nom_min = n.stores.e_nom_min.add(additional_stores, fill_value = 0)
    #display(n.stores.e_nom.sum())


    # ------- LINKS -------
    additional_links = m.links.p_nom_opt - m.links.p_nom
    additional_links
    n.links.p_nom = n.links.p_nom.add(additional_links, fill_value = 0)
    n.links.p_nom_min = n.links.p_nom_min.add(additional_links, fill_value = 0)
    #display(n.links.p_nom.sum())


    # ------- DECOM ---------
    #for index,value in decom[year].items():
    #    n.generators.loc[index, 'p_nom'] = n.generators.loc[index].p_nom - value
    #    n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min - value

    # -------- EXPANSION ----------
    # for index,value in expansion[year].items():
    #     n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min + value

    # ------- HYDRO-DECOM-------- # is done seperately, because the  code is different
    #if year == 2032:
    #    n.storage_units.loc['BO 2 hydro','p_nom'] = n.storage_units.loc['BO 2 hydro'].p_nom - 2.55

    # ------- SAVING ----------
    n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_27_ec_lcopt_Co2L-1H.nc')

## Snakemake process

In [32]:
# Function that renames the results file and moves it to the scenario folder

def rename_results_file(year):
    current_location = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/results/networks/'
    old_filename = "elec_s_27_ec_lcopt_Co2L-1H.nc"
    new_filename = f"{scen}_{year}.nc"
    new_location = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/'

    # Create the full paths for the old and new files
    old_file_path = os.path.join(current_location, old_filename)
    new_file_path = os.path.join(new_location, new_filename)

    # Rename the file
    os.rename(old_file_path, new_file_path)


In [34]:
# determine scenario
scen_folder = 'results_RC_10'
scen = 'RC_10'

# !!!! Make sure the upscaling, emission constraints and decomissioning is run!!!

# run for loop over all years
for year in range(2021, 2036):
    n = read_network_file()
    if year == 2021:
        !snakemake -j 1 solve_all_networks
        rename_results_file(year)
    else:
        yearly_changes(n,year)
        !snakemake -j 1 solve_all_networks 
        rename_results_file(year)  

# And you're DONE!!! :D


c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found


INFO:pypsa.io:Imported network elec_s_27_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.

2022


INFO:pypsa.io:Imported network RC_10_2021.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Exported network elec_s_27_ec_lcopt_Co2L-1H.nc has global_constraints, generators, storage_units, links, stores, loads, buses, lines, carriers
C:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(
Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

Select jobs t

...

INFO:pypsa.io:Imported network elec_s_27_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.

2035


INFO:pypsa.io:Imported network RC_10_2034.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Exported network elec_s_27_ec_lcopt_Co2L-1H.nc has global_constraints, generators, storage_units, links, stores, loads, buses, lines, carriers


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
Read LP format model from file C:\Users\noraky\AppData\Local\Temp\pypsa-problem-_98f88zk.lp
Reading time = 9.09 seconds
obj: 4879348 rows, 2295304 columns, 9904241 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 4 threads

Optimize a model with 4879348 rows, 2295304

C:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(
Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

Select jobs to execute...

INFO:snakemake.logging:
[Mon Nov 13 17:08:47 2023]
INFO:snakemake.logging:[Mon Nov 13 17:08:47 2023]
rule solve_network:
    input: networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    output: results/networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    log: logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H

# Changes to the Network

Here all the general changes (except for expandabels, that's at the end of this file) that can be made to the model are shown. If more than the two last rules of the model are run, some of these changes may have to be done again. Use the "Checks" section to check your current network file. 

### Add Biomass in Node 0 and 3

The nodes may need to be adapted, depending on your model

In [ ]:
n.add("Generator",
            'BO 0 biomass',
            bus='BO 0',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)


n.add("Generator",
            'BO 2 biomass',
            bus='BO 2',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)

### Add ROR 

Nodes may vary

In [ ]:
n.storage_units

In [ ]:
n.add("Generator",
            'BO 2 ror',
            bus='BO 2',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

n.add("Generator",
            'BO 3 ror',
            bus='BO 3',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

### Biomas Max Capacity

The nodes may vary, the larger biomass max capacity is located in the Oriental zone

In [ ]:
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'biomass':
        if index == 'BO 1 biomass':
        # Update the p_max_pu value to 0.64
            n.generators.at[index, 'p_nom_max'] = 357.13
        else:
            n.generators.at[index, 'p_nom_max'] = 21.01

n.generators.p_nom_max

### Battery max capacity

In [ ]:
# discharger capacity

for index, row in n.links.iterrows():
    if row['carrier'] == 'battery discharger':
        n.links.at[index, 'p_nom_max'] = 225

display(n.links.p_nom_max)


# storage capacity

for index, row in n.stores.iterrows():
    if row['carrier'] == 'battery':
        n.stores.at[index, 'e_nom_max'] = 1350

n.stores.e_nom_max

### Capacity factors

With the new addition to model where the annual capacity factors can be set, this is not necessary anymore.

In [6]:
# Check the current status
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       1.0
OCGT       1.0
biomass    1.0
oil        1.0
onwind     1.0
ror        1.0
solar      1.0
Name: p_max_pu, dtype: float64

In [ ]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'OCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 # 0.08 #0.64
    if row['carrier'] == 'CCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 #0.45 #0.64

    if row['carrier'] == 'biomass':
        # Update the p_max_pu value to 0.72
        n.generators.at[index, 'p_max_pu'] = 0.72 #0.32 #0.72 for the future 

        
n.generators.groupby('carrier').p_max_pu.mean()

# Save elec_s_4_ec_lcopt_Co2L-1H file

If you made changes to network file at the beginning, they need to be saved again. This code can be used for that. 

In [10]:
n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_27_ec_lcopt_Co2L-1H.nc')

INFO:pypsa.io:Exported network elec_s_27_ec_lcopt_Co2L-1H.nc has global_constraints, generators, storage_units, links, stores, loads, buses, lines, carriers


<xarray.Dataset>
Dimensions:                               (snapshots: 8760,
                                           investment_periods: 0,
                                           global_constraints_i: 1,
                                           generators_i: 73,
                                           generators_t_p_max_pu_i: 56,
                                           storage_units_i: 5,
                                           storage_units_t_inflow_i: 5,
                                           links_i: 56, stores_i: 28,
                                           loads_i: 28, loads_t_p_set_i: 28,
                                           buses_i: 56, lines_i: 52,
                                           carriers_i: 11)
Coordinates: (12/14)
  * snapshots                             (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                    (investment_periods) int64 
  * global_constraints_i                  (global_constraints_i) object 'CO2L...
  * generators_i                          (generators_i) object 'BO0 0 OCGT' ...
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 'B...
  * storage_units_i                       (storage_units_i) object 'BO0 1 hyd...
    ...                                    ...
  * stores_i                              (stores_i) object 'BO0 0 battery' ....
  * loads_i                               (loads_i) object 'BO0 0' ... 'BO1 0'
  * loads_t_p_set_i                       (loads_t_p_set_i) object 'BO0 0' .....
  * buses_i                               (buses_i) object 'BO0 0' ... 'BO1 0...
  * lines_i                               (lines_i) object '1' '2' ... '51' '52'
  * carriers_i                            (carriers_i) object 'OCGT' ... 'bat...
Data variables: (12/81)
    snapshots_snapshot                    (snapshots) datetime64[ns] 2013-01-...
    snapshots_objective                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 
    investment_periods_years              (investment_periods) float64 
    ...                                    ...
    lines_s_nom_extendable                (lines_i) bool True True ... True True
    lines_s_nom_min                       (lines_i) float64 569.2 ... 853.8
    lines_build_year                      (lines_i) int64 0 0 0 0 0 ... 0 0 0 0
    carriers_co2_emissions                (carriers_i) float64 0.187 ... 0.0
    carriers_color                        (carriers_i) object '#d35050' ... '...
    carriers_nice_name                    (carriers_i) object 'Open-Cycle Gas...
Attributes:
    network_name:           
    network_pypsa_version:  0.24.0
    network_srid:           4326
    meta:                   {"version": "0.2.2", "tutorial": false, "logging"...

# Checks

If you don't know the setting of your current network file, use the following cells to check it. 

In [11]:
# Which generators do exist?

n.generators

,p_nom_max,weight,p_nom,capital_cost,efficiency,p_nom_min,marginal_cost,bus,p_nom_extendable,carrier,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
BO0 0 OCGT,inf,0.000000,369.133797,47234.561404,0.390,369.133797,58.384615,BO0 0,True,OCGT,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO0 0 onwind,3.950012e+03,474.833593,0.005269,109295.569252,1.000,0.005269,0.015000,BO0 0,True,onwind,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO0 0 solar,5.029602e+03,1007.262554,0.032581,50855.958267,1.000,0.032581,0.010000,BO0 0,True,solar,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO0 1 onwind,3.737247e+04,984.122775,0.001580,109295.569252,1.000,0.001580,0.015000,BO0 1,True,onwind,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO0 1 ror,inf,7.540000,7.540000,270940.715283,0.900,0.000000,0.000000,BO0 1,False,ror,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BO0 9 solar,2.961211e+04,4849.099325,5.000608,50855.958267,1.000,5.000608,0.010000,BO0 9,True,solar,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO1 0 biomass,inf,0.000000,600.344050,278015.359506,0.468,600.344050,14.957265,BO1 0,True,biomass,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
BO1 0 oil,inf,0.000000,94.720859,38234.561404,0.393,94.720859,130.226463,BO1 0,True,oil,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


In [12]:
# Which generators are currently extendable?

n.generators.p_nom_extendable


Generator
BO0 0 OCGT        True
BO0 0 onwind      True
BO0 0 solar       True
BO0 1 onwind      True
BO0 1 ror        False
                 ...  
BO0 9 solar       True
BO1 0 biomass     True
BO1 0 oil         True
BO1 0 onwind      True
BO1 0 solar       True
Name: p_nom_extendable, Length: 73, dtype: bool

In [13]:
# Check the current status of the cp
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       1.0
OCGT       1.0
biomass    1.0
oil        1.0
onwind     1.0
ror        1.0
solar      1.0
Name: p_max_pu, dtype: float64

In [14]:
# Which generators have which expansion limit?

n.generators.p_nom_max

Generator
BO0 0 OCGT                inf
BO0 0 onwind     3.950012e+03
BO0 0 solar      5.029602e+03
BO0 1 onwind     3.737247e+04
BO0 1 ror                 inf
                     ...     
BO0 9 solar      2.961211e+04
BO1 0 biomass             inf
BO1 0 oil                 inf
BO1 0 onwind     8.039063e+05
BO1 0 solar      3.963594e+05
Name: p_nom_max, Length: 73, dtype: float64

In [15]:
# Which links have which expansion limit?

n.links.p_nom_max

Link
BO0 0 battery charger        inf
BO0 1 battery charger        inf
BO0 10 battery charger       inf
BO0 11 battery charger       inf
BO0 12 battery charger       inf
BO0 13 battery charger       inf
BO0 14 battery charger       inf
BO0 15 battery charger       inf
BO0 16 battery charger       inf
BO0 17 battery charger       inf
BO0 18 battery charger       inf
BO0 19 battery charger       inf
BO0 2 battery charger        inf
BO0 20 battery charger       inf
BO0 21 battery charger       inf
BO0 22 battery charger       inf
BO0 23 battery charger       inf
BO0 24 battery charger       inf
BO0 25 battery charger       inf
BO0 26 battery charger       inf
BO0 3 battery charger        inf
BO0 4 battery charger        inf
BO0 5 battery charger        inf
BO0 6 battery charger        inf
BO0 7 battery charger        inf
BO0 8 battery charger        inf
BO0 9 battery charger        inf
BO1 0 battery charger        inf
BO0 0 battery discharger     inf
BO0 1 battery discharger     inf
BO0 1

In [16]:
# What is the sum of the demand in the network?

round(n.loads_t.p_set.sum().sum()/1000000,8)

13.62275428

In [17]:
# Is there any installed storage capacity?

n.stores.e_nom

Store
BO0 0 battery       0.452564
BO0 1 battery       0.451968
BO0 10 battery      0.453017
BO0 11 battery      0.452332
BO0 12 battery      0.452261
BO0 13 battery      0.450394
BO0 14 battery      0.452442
BO0 15 battery      0.452890
BO0 16 battery      0.450836
BO0 17 battery      0.452975
BO0 18 battery      0.451833
BO0 19 battery      0.451031
BO0 2 battery       0.450206
BO0 20 battery      0.452312
BO0 21 battery      0.451750
BO0 22 battery      0.451948
BO0 23 battery      0.453347
BO0 24 battery      0.452568
BO0 25 battery      0.454331
BO0 26 battery      0.451011
BO0 3 battery       0.450332
BO0 4 battery       0.449162
BO0 5 battery       0.451988
BO0 6 battery       0.451338
BO0 7 battery       0.452622
BO0 8 battery       0.452563
BO0 9 battery       0.451606
BO1 0 battery     583.359042
Name: e_nom, dtype: float64

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
Read LP format model from file C:\Users\noraky\AppData\Local\Temp\pypsa-problem-kprx9mzd.lp
Reading time = 9.93 seconds
obj: 5422469 rows, 2496806 columns, 11629945 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 4 threads

Optimize a model with 5422469 rows, 249680

C:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(
Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

Select jobs to execute...

INFO:snakemake.logging:
[Mon Nov 13 11:48:17 2023]
INFO:snakemake.logging:[Mon Nov 13 11:48:17 2023]
rule solve_network:
    input: networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    output: results/networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    log: logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H

In [18]:
# What is the current emission limit?

n.global_constraints.constant.loc['CO2Limit']

846428.5714285714

In [19]:
# Which stores have a maximum expansion limit?

n.stores.e_nom_max

Store
BO0 0 battery     inf
BO0 1 battery     inf
BO0 10 battery    inf
BO0 11 battery    inf
BO0 12 battery    inf
BO0 13 battery    inf
BO0 14 battery    inf
BO0 15 battery    inf
BO0 16 battery    inf
BO0 17 battery    inf
BO0 18 battery    inf
BO0 19 battery    inf
BO0 2 battery     inf
BO0 20 battery    inf
BO0 21 battery    inf
BO0 22 battery    inf
BO0 23 battery    inf
BO0 24 battery    inf
BO0 25 battery    inf
BO0 26 battery    inf
BO0 3 battery     inf
BO0 4 battery     inf
BO0 5 battery     inf
BO0 6 battery     inf
BO0 7 battery     inf
BO0 8 battery     inf
BO0 9 battery     inf
BO1 0 battery     inf
Name: e_nom_max, dtype: float64

In [20]:
n.buses

,v_nom,country,x,y,control,generator,carrier,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,sub_network
Bus,,,,,,,,,,,,,
BO0 0,220.0,BO,-63.203950,-17.749500,Slack,BO0 0 OCGT,AC,,,1.0,0.0,inf,
BO0 1,220.0,BO,-65.734400,-19.558733,PQ,,AC,,,1.0,0.0,inf,
BO0 10,220.0,BO,-64.305800,-19.321700,PQ,,AC,,,1.0,0.0,inf,
BO0 11,220.0,BO,-68.131500,-16.354360,PQ,,AC,,,1.0,0.0,inf,
BO0 12,220.0,BO,-65.742100,-21.432100,PQ,,AC,,,1.0,0.0,inf,
BO0 13,220.0,BO,-66.220600,-17.569700,PQ,,AC,,,1.0,0.0,inf,
BO0 14,220.0,BO,-67.795050,-16.342950,PQ,,AC,,,1.0,0.0,inf,
BO0 15,220.0,BO,-65.216400,-18.999000,PQ,,AC,,,1.0,0.0,inf,
BO0 16,220.0,BO,-63.233675,-17.444700,PQ,,AC,,,1.0,0.0,inf,


# Restoring to 2021

When you reached your final year, you can easily restore your network to the year 2021. It is recommended, though, to save the initial 2021 file at the beginning. 

## Demand

In [ ]:
year = 'back'
upscaling_factor = scale_demand[year]   # 0.569269674



n.loads_t.p_set = n.loads_t.p_set * upscaling_factor

year = 2021

In [ ]:
n.loads_t.p_set.sum().sum()

## Emission limit

In [ ]:
n.global_constraints.constant = 1000000000 #emission_limit[2021]
n.global_constraints.constant

## Expansions

In [ ]:
solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/results_NS/NS_2021.nc'
m = pypsa.Network(solved_network)

n.generators.p_nom = m.generators.p_nom
n.generators.p_nom

In [ ]:
n.generators.p_nom_min = m.generators.p_nom_min
n.generators.p_nom_min

In [ ]:
# new biomass to 0
for index, row in n.generators.iterrows():
    if index == 'BO 0 biomass':
        n.generators.at[index, 'p_nom'] = 0
    if index == 'BO 2 biomass':
            n.generators.at[index, 'p_nom'] = 0
n.generators.p_nom

In [ ]:
n.stores.e_nom = 0

n.stores.e_nom.sum()

In [ ]:
n.links.p_nom = 0
n.links.p_nom.sum()

# Changing the expandabel carriers

This can also be done in the config file, but doing it here saves you from running the model again and have to do all the other changes again as well.

### RES to TRUE

In [ ]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'biomass':
        
        n.generators.at[index, 'p_nom_extendable'] = True 
    if row['carrier'] == 'solar':
        
        n.generators.at[index, 'p_nom_extendable'] = True 

    if row['carrier'] == 'onwind':
       
        n.generators.at[index, 'p_nom_extendable'] = True 

### Fossils to FALSE

In [ ]:
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'OCGT':
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'CCGT':
        
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'oil':
       
        n.generators.at[index, 'p_nom_extendable'] = False 

n.generators.p_nom_extendable